##### TwinBert https://arxiv.org/pdf/2002.06275v1.pdf

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 13.8 MB/s 
     |████████████████████████████████| 7.6 MB 49.6 MB/s 
     |████████████████████████████████| 163 kB 74.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### The dataset is Quora questions pairs dataset. Ideally in the paper the authors have trained the model to use it as a backend for a sponsored search engine, to delivers ads alongside the organic search results.

In [15]:
df = pd.read_csv("/content/drive/MyDrive/machine_learning/datasets/im_query2product/train.csv", sep=',')   # Dataset : https://www.kaggle.com/c/quora-question-pairs

In [31]:
# df = df.drop(978549)

#df[df['train_label'].isna()]
series = df.loc[:,"query"]

In [32]:
series[0]

'1kg tur dal'

## Dataset Loader for the Siamese Network 

In [39]:
class SiameseNetworkDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.question1 = dataframe.loc[:,"query"]
        self.question2 = dataframe.loc[:,"product_name"]
        self.targets = dataframe.loc[:,"train_label"]
        self.max_len = max_len
        print(self.question1[4])
        
    def __len__(self):
        return len(self.data)
    
    
    def tokenize(self,input_text):
        input_text = " ".join(input_text.split())

        inputs = self.tokenizer.encode_plus(
            input_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        return ids,mask,token_type_ids

    def __getitem__(self, index):
        ids1,mask1,token_type_ids1 = self.tokenize(str(self.question1[index]))
        ids2,mask2,token_type_ids2 = self.tokenize(str(self.question2[index]))
        


        return {
            'ids': [torch.tensor(ids1, dtype=torch.long),torch.tensor(ids2, dtype=torch.long)],
            'mask': [torch.tensor(mask1, dtype=torch.long),torch.tensor(mask2, dtype=torch.long)],
            'token_type_ids': [torch.tensor(token_type_ids1, dtype=torch.long),torch.tensor(token_type_ids2, dtype=torch.long)],
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }
   

## TwinBert architecture

In [40]:
class TwinBert(nn.Module):
    def __init__(self):
        super(TwinBert, self).__init__()
        self.model = transformers.BertModel.from_pretrained('bert-base-uncased')
    def forward_once(self, ids, mask, token_type_ids):
        _, output= self.model(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        return output
    def forward(self, ids, mask, token_type_ids):
        output1 = self.forward_once(ids[0],mask[0], token_type_ids[0])
        output2 = self.forward_once(ids[1],mask[1], token_type_ids[1])
        return output1,output2
        

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model = TwinBert()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TwinBert(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [41]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [42]:
train_size = 0.8
df = df.reset_index(drop=True)
train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
testing_set = SiameseNetworkDataset(test_dataset, tokenizer, MAX_LEN)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = SiameseNetworkDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = SiameseNetworkDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1875264, 3)
TRAIN Dataset: (1500211, 3)
TEST Dataset: (375053, 3)
kabab masala powder
yippee maggie


In [43]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# Loss Function
### A contrastive loss function that takes cosine similarity as a metric to measure the distance.

In [44]:
class CosineContrastiveLoss(nn.Module):
    def __init__(self, margin=0.4):
        super(CosineContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        cos_sim = F.cosine_similarity(output1, output2)
        loss_cos_con = torch.mean((1-label) * torch.div(torch.pow((1.0-cos_sim), 2), 4) +
                                    (label) * torch.pow(cos_sim * torch.lt(cos_sim, self.margin), 2))
        return loss_cos_con

In [45]:
criterion = CosineContrastiveLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0005 )

# Training

In [ ]:
for epoch in range(EPOCHS):
  model.train()
  for _,data in enumerate(training_loader, 0):
      ids,mask,token_type_ids = data['ids'],data['mask'],data['token_type_ids'] 
      targets = data['targets'].to(device, dtype = torch.float)
      ids = [ids[0].to(device, dtype = torch.long),ids[1].to(device, dtype = torch.long)]
      mask = [mask[0].to(device, dtype = torch.long),mask[1].to(device, dtype = torch.long)]
      token_type_ids = [token_type_ids[0].to(device, dtype = torch.long),token_type_ids[1].to(device, dtype = torch.long)]
      output1,output2 = model(ids, mask, token_type_ids)
      optimizer.zero_grad()
      loss = criterion(output1,output2,targets)
      if _%1==0:
          print(f'Step: {_}, Epoch: {epoch}, Loss:  {loss.item()}')
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [48]:
torch.save(model.state_dict(), '/content/drive/MyDrive/machine_learning/im_query2product/model_dict.pt')
torch.save(model, '/content/drive/MyDrive/machine_learning/im_query2product/model.pt')



# Validation

In [54]:
# train_dataset=df.sample(frac=train_size,random_state=200).reset_index(drop=True)
small_test_dataset=test_dataset.sample(frac = 0.01, random_state=200).reset_index(drop=True)
small_testing_set = SiameseNetworkDataset(small_test_dataset, tokenizer, MAX_LEN)

small_testing_loader = DataLoader(small_testing_set, **test_params)

banana pudding cake


In [56]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    count = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids,mask,token_type_ids = data['ids'],data['mask'],data['token_type_ids'] 
            targets = data['targets'].to(device, dtype = torch.float)
            ids = [ids[0].to(device, dtype = torch.long),ids[1].to(device, dtype = torch.long)]
            mask = [mask[0].to(device, dtype = torch.long),mask[1].to(device, dtype = torch.long)]
            token_type_ids = [token_type_ids[0].to(device, dtype = torch.long),token_type_ids[1].to(device, dtype = torch.long)]
            targets = data['targets'].to(device, dtype = torch.float)
            output1,output2 = model(ids, mask, token_type_ids)
            cos_sim = F.cosine_similarity(output1, output2)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(cos_sim).cpu().detach().numpy().tolist())
            count += 1
            if (count%100 == 0):
              print(count)
    return fin_outputs, fin_targets

In [57]:
outputs, targets = validation(small_testing_loader)
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


100
200
300
400
500
600
700
800
900
Accuracy Score = 0.2751266328978939
F1 Score (Micro) = 0.2751266328978939
F1 Score (Macro) = 0.21576416475015683


In [59]:
print(outputs.shape)

(3751,)
